### Step 3: Data Cleaning & Processing

---

#### 3.1 Stage 1 - Schema & Type Audit  
The dataset schema and attributes were inspected, technical/index columns were dropped, and data types along with missing values were carefully examined.  

#### 3.2 Stage 2 - Data Description  
Statistical summaries of the dataset were generated, the distributions of numerical features were reviewed, and unique values within categorical features were explored.  

#### 3.3 Stage 3 - Data Cleaning & Processing  
Redundant columns were removed, data types were converted into appropriate formats, missing values were handled, categorical fields were normalized (e.g., Yes/No mapped to 0/1), and the dataset was prepared for modeling.  

#### 3.4 Stage 4 - Exploratory Data Analysis (EDA)  
Selected numerical and categorical features were analyzed, correlations were examined, and visual plots were generated to uncover patterns within the dataset. 

#### 3.5 Stage 5 - Feature Engineering  
New derived variables were created, categorical fields were encoded, and feature transformations were applied to enhance data readiness.  

#### 3.6 Stage 6 - Save Processed Dataset  
The cleaned and processed dataset was exported into a CSV file, ensuring reproducibility and availability for subsequent modeling tasks. 

---

### Stage 0 - Setup & Load (Reproducibility + Import)

**Role:**  
Create a clean, reproducible environment; load data once identifying its encoding; centralize configuration (column lists, thresholds).

**Outcome:**  
A DataFrame and shared settings used by every subsequent step.

In [1]:
import pandas as pd
import numpy as np
import chardet
from collections import defaultdict

# Display settings (optional)
pd.set_option('display.max_columns', None)

# ---- Paths ----
RAW_PATH = 'diabetes_diagnosis.csv'
CLEAN_PATH = 'diabetes_diagnosis_clean.csv'

In [2]:
# Read a small portion of the file to understand the encoding
with open(RAW_PATH, 'rb') as f:
    result = chardet.detect(f.read(10000))
    print("Encoding of the provided .csv file is:",result['encoding'])

# Load the csv file in a Dataframe
try:
    df = pd.read_csv(
        RAW_PATH,
        encoding=result['encoding'],         
    )
    print("CSV file loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at path {RAW_PATH}")
except pd.errors.ParserError as e:
    print(f"Parsing error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Quick peek at the dataset
print("\nNumber of (Rows, Columns) in the dataframe:",df.shape, "\n")
df.head(3)

Encoding of the provided .csv file is: ascii
CSV file loaded successfully.

Number of (Rows, Columns) in the dataframe: (264802, 23) 



,Unnamed: 0,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,AnyHealthcare,NoDocbcCost,GeneralHealth,Mental (days),Physical (days),DiffWalk,Sex,Age,Education,Income,Diabetes,BloodPressure,Cholesterol,Alcoholic
0,0,NaN,40.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,15.0,NaN,NaN,63.0,High school graduate,"$73,106",No,Yes,Yes,No
1,1,NaN,25.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,Good,NaN,NaN,NaN,Female,54.0,College graduate,"$22,322",No,No,No,No
2,2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1.0,Poor,30.0,NaN,1.0,NaN,NaN,High school graduate,"$29,097",No,Yes,Yes,NaN


In [3]:
df.describe() # Stats about the dataset

,Unnamed: 0,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,AnyHealthcare,NoDocbcCost,Mental (days),Physical (days),DiffWalk,Age
count,264802.000000,113865.000000,177418.000000,145642.000000,105921.000000,193306.000000,76793.000000,103273.000000,95329.000000,82089.000000,66201.000000,135050.000000,121809.000000,121809.000000,201250.000000
mean,132400.500000,0.963931,28.378631,0.448078,0.038566,0.089190,0.769328,0.626359,0.820590,0.954367,0.079757,3.072817,4.059388,0.160694,57.350857
std,76441.897328,0.186463,17.649248,2.400180,0.192560,0.285019,0.421266,2.386877,0.383697,0.208690,0.270919,8.627261,11.084963,0.367250,51.687159
min,0.000000,0.000000,-46.000000,-10.000000,0.000000,0.000000,0.000000,-10.000000,0.000000,0.000000,0.000000,-20.000000,-30.000000,0.000000,-163.000000
25%,66200.250000,1.000000,24.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,46.000000
50%,132400.500000,1.000000,27.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,59.000000
75%,198600.750000,1.000000,32.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000,3.000000,0.000000,69.000000
max,264801.000000,1.000000,101.000000,11.000000,1.000000,1.000000,1.000000,11.000000,1.000000,1.000000,1.000000,30.000000,33.000000,1.000000,278.000000


### 3.1 Stage 1 - Schema & Type Audit

**Role:**  
Ensure the dataset has the expected columns, drop purely technical columns, and set the right data types.

**Key Actions:**
- Drop `Unnamed: 0` (row index artifact).  
- Inspect `dtypes` and coerce obvious types (e.g., numbers stored as strings).  
- Keep a quick data dictionary for later reference. 

In [4]:
# Drop technical/index-like columns if present
to_drop = [c for c in df.columns if c.lower().startswith('unnamed')]
df = df.drop(columns=to_drop, errors='ignore')

expected_cols = [
    'CholCheck','BMI','Smoker','Stroke','HeartDiseaseorAttack',
    'PhysActivity','Fruits','Veggies','AnyHealthcare','NoDocbcCost',
    'GeneralHealth','Mental (days)','Physical (days)','DiffWalk',
    'Sex','Age','Education','Income','Diabetes',
    'BloodPressure','Cholesterol','Alcoholic'
]
missing_from_expected = [c for c in expected_cols if c not in df.columns]
print("Missing columns from expected:", missing_from_expected)

# Snapshot of the Dataframe
df_info = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'n_missing': df.isna().sum(),
    'missing_%': (df.isna().sum() / len(df) * 100).round(2)
})

df_info

Missing columns from expected: []


,dtype,n_missing,missing_%
CholCheck,float64,150937,57.0
BMI,float64,87384,33.0
Smoker,float64,119160,45.0
Stroke,float64,158881,60.0
HeartDiseaseorAttack,float64,71496,27.0
PhysActivity,float64,188009,71.0
Fruits,float64,161529,61.0
Veggies,float64,169473,64.0
AnyHealthcare,float64,182713,69.0
NoDocbcCost,float64,198601,75.0


### 3.2 Stage 2 - Standardize Raw Values & Fix Sentinels

**Role:**  
Convert inconsistent encodings and eliminate “impossible” or placeholder values so downstream stats aren’t biased.

**Key Actions:**
- Parse **Income** (remove `$`, `,`, etc.).
- Convert "Yes/No" values from **BloodPressure**, **Cholesterol** & **Alcholic** into 1 or 0
- Replace sentinel values (e.g., `-10, -20, -30, -46, -163`) with `NaN`.  
- Apply domain-based validity ranges:
  - `Age < 0 or > 120 → NaN`  
  - `BMI < 10 or > 90 → NaN`  
  - `Mental (days), Physical (days)` outside `0–30 → NaN`

In [5]:
# Utility: parse currency-like strings to numeric
def parse_currency_to_float(s):
    if pd.isna(s):
        return np.nan
    # Remove $ , and spaces
    s = str(s).replace('$', '').replace(',', '').strip()
    # Empty or non-numeric -> NaN
    try:
        return float(s)
    except ValueError:
        return np.nan

# 2.1 Income to numeric
if 'Income' in df.columns:
    df['Income_num'] = df['Income'].apply(parse_currency_to_float)

In [6]:
# 2.2 Convert Yes/No columns to 0/1
yn_map = {
    'yes': 1.0, 'y': 1.0, 'true': 1.0, '1': 1.0,
    'no': 0.0, 'n': 0.0, 'false': 0.0, '0': 0.0
}

def yes_no_to_float(s):
    if pd.isna(s):
        return np.nan
    s_norm = str(s).strip().lower()
    return yn_map.get(s_norm, np.nan)  # anything else -> NaN

for col in ['BloodPressure', 'Cholesterol', 'Alcoholic']:
    if col in df.columns:
        df[col] = df[col].apply(yes_no_to_float).astype('float64')

In [7]:
# 2.3 Replace global sentinel values with NaN for numeric-like columns
sentinels = {-10, -20, -30, -46, -163}
numeric_like_cols = ['BMI','Smoker','Stroke','HeartDiseaseorAttack',
                     'PhysActivity','Fruits','Veggies','AnyHealthcare',
                     'NoDocbcCost','Mental (days)','Physical (days)',
                     'DiffWalk','Age','Income_num','CholCheck']

for col in numeric_like_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df.loc[df[col].isin(sentinels), col] = np.nan

In [8]:
# 2.4 Domain-based validity ranges
if 'Age' in df.columns:
    df.loc[(df['Age'] < 0) | (df['Age'] > 120), 'Age'] = np.nan

if 'BMI' in df.columns:
    df.loc[(df['BMI'] < 10) | (df['BMI'] > 90), 'BMI'] = np.nan

for day_col in ['Mental (days)','Physical (days)']:
    if day_col in df.columns:
        df.loc[(df[day_col] < 0) | (df[day_col] > 30), day_col] = np.nan

In [9]:
df.describe()

,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,AnyHealthcare,NoDocbcCost,Mental (days),Physical (days),DiffWalk,Age,BloodPressure,Cholesterol,Alcoholic,Income_num
count,113865.000000,168525.000000,142020.000000,105921.000000,193306.000000,76793.000000,100703.000000,95329.000000,82089.000000,66201.000000,131678.000000,115625.000000,121809.000000,191165.000000,219786.000000,225082.000000,209194.000000,246266.000000
mean,0.963931,28.379522,0.714540,0.038566,0.089190,0.769328,0.897550,0.820590,0.954367,0.079757,3.663664,4.226785,0.160694,57.369518,0.428831,0.423646,0.056445,38078.621864
std,0.186463,6.562184,1.747211,0.192560,0.285019,0.421266,1.699186,0.383697,0.208690,0.270919,7.896450,8.708876,0.367250,15.732494,0.494910,0.494137,0.230780,33810.553811
min,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,24.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,47.000000,0.000000,0.000000,0.000000,14979.000000
50%,1.000000,27.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,59.000000,0.000000,0.000000,0.000000,25017.500000
75%,1.000000,31.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000,3.000000,0.000000,68.000000,1.000000,1.000000,0.000000,49980.000000
max,1.000000,90.000000,11.000000,1.000000,1.000000,1.000000,11.000000,1.000000,1.000000,1.000000,30.000000,30.000000,1.000000,90.000000,1.000000,1.000000,1.000000,149993.000000


### 3.3 Stage 3 - Normalize Binary & Categorical Variables

**Role:**  
Make categories consistent and machine-readable while preserving human-readable originals where useful.

**Key Actions:**

- **Binary health flags**  
  (`CholCheck, Smoker, Stroke, HeartDiseaseorAttack, PhysActivity, Fruits, Veggies, AnyHealthcare, NoDocbcCost, DiffWalk, BloodPressure, Cholesterol, Alcoholic`)  
  - Convert any positive value → `1`,  
  - `0` stays `0`,  
  - invalid/other → `NaN`.

- **Ordinal encodings** (preserve order):  
  - `GeneralHealth → Excellent > Very Good > Good > Fair > Poor`  
  - `Education → Elementary < Some high school < High school graduate < Some college < College graduate`  

- **Nominal simplifications:**  
  - `Sex: Male → 1, Female → 0`  
  - `Diabetes`: keep original categories (`No, Prediabetes, Diabetes`)  
  - Add encoded target column:  
    - `No = 0`, `Prediabetes = 1`, `Diabetes = 2`  


In [10]:
# 3.1 Binary columns (convert > 0 -> 1, 0 stays 0, others -> NaN)
binary_cols = [
    'CholCheck','Smoker','Stroke','HeartDiseaseorAttack','PhysActivity',
    'Fruits','Veggies','AnyHealthcare','NoDocbcCost','DiffWalk',
    'BloodPressure','Cholesterol','Alcoholic'
]
for col in binary_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df.loc[df[col] > 0, col] = 1.0
        df.loc[df[col] == 0, col] = 0.0 # values < 0 already set to NaN in stage 2

In [11]:
# 3.2 Ordinal encodings
# GeneralHealth: Excellent(5) > Very Good(4) > Good(3) > Fair(2) > Poor(1)
gh_map = {'Excellent':5, 'Very Good':4, 'Good':3, 'Fair':2, 'Poor':1}
if 'GeneralHealth' in df.columns:
    df['GeneralHealth_ord'] = df['GeneralHealth'].map(gh_map)

# Education: Elementary(1) < Some high school(2) < High school graduate(3) < Some college(4) < College graduate(5)
edu_map = {
    'Elementary':1,
    'Some high school':2,
    'High school graduate':3,
    'Some college':4,
    'College graduate':5
}
if 'Education' in df.columns:
    df['Education_ord'] = df['Education'].map(edu_map)

In [12]:
# 3.3 Sex: Male=1, Female=0
sex_map = {'Male':1, 'Female':0}
if 'Sex' in df.columns:
    df['Sex_bin'] = df['Sex'].map(sex_map)

In [13]:
# 3.4 Target encoding (keep original + numeric label)
# Diabetes_label: No=0, Prediabetes=1, Diabetes=2
target_map = {'No':0, 'Prediabetes':1, 'Diabetes':2}
if 'Diabetes' in df.columns:
    df['Diabetes_label'] = df['Diabetes'].map(target_map)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264802 entries, 0 to 264801
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CholCheck             113865 non-null  float64
 1   BMI                   168525 non-null  float64
 2   Smoker                142020 non-null  float64
 3   Stroke                105921 non-null  float64
 4   HeartDiseaseorAttack  193306 non-null  float64
 5   PhysActivity          76793 non-null   float64
 6   Fruits                100703 non-null  float64
 7   Veggies               95329 non-null   float64
 8   AnyHealthcare         82089 non-null   float64
 9   NoDocbcCost           66201 non-null   float64
 10  GeneralHealth         195954 non-null  object 
 11  Mental (days)         131678 non-null  float64
 12  Physical (days)       115625 non-null  float64
 13  DiffWalk              121809 non-null  float64
 14  Sex                   195954 non-null  object 
 15  

In [15]:
df.head(3)

,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,AnyHealthcare,NoDocbcCost,GeneralHealth,Mental (days),Physical (days),DiffWalk,Sex,Age,Education,Income,Diabetes,BloodPressure,Cholesterol,Alcoholic,Income_num,GeneralHealth_ord,Education_ord,Sex_bin,Diabetes_label
0,NaN,40.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,15.0,NaN,NaN,63.0,High school graduate,"$73,106",No,1.0,1.0,0.0,73106.0,NaN,3.0,NaN,0.0
1,NaN,25.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,Good,NaN,NaN,NaN,Female,54.0,College graduate,"$22,322",No,0.0,0.0,0.0,22322.0,3.0,5.0,0.0,0.0
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1.0,Poor,30.0,NaN,1.0,NaN,NaN,High school graduate,"$29,097",No,1.0,1.0,NaN,29097.0,1.0,3.0,NaN,0.0


### 3.4 Stage 4 - Missing Data Strategy (Indicators + Imputation)

**Role:**  
Handle missingness without leaking information; retain signal in *why missing* via flags.

**Key Actions:**
- Create missing-indicator columns (e.g., `PhysActivity_missing`) for very sparse features (≥50% missing).  
- Impute numeric features → **median** (robust to outliers).  
- Impute binary/ordinal/nominal features → **mode** (most frequent).  
- Optionally refine later (e.g., stratified imputation or KNN).  

In [16]:
df = df.dropna(subset = ['Diabetes']) # Drop all the missing values from the target column

In [17]:
# 4.1 Add missingness indicators for very sparse features (>= 50% missing)
sparse_threshold = 0.50
missing_ratio = df.isna().mean()
very_sparse_cols = missing_ratio[missing_ratio >= sparse_threshold].index.tolist()

# Only add indicators for columns that exist & are meaningful to flag
indicator_exclude = set(['Diabetes', 'Diabetes_label'])
for col in very_sparse_cols:
    if col not in indicator_exclude:
        df[f'{col}_missing'] = df[col].isna().astype(int)

# (Optional) Recompute missingness to confirm
# post_missing = df.isna().mean().sort_values(ascending=False).head(10)

In [18]:
# 4.2 Impute numeric features with median
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Don't impute the target label
numeric_cols = [c for c in numeric_cols if c != 'Diabetes_label']

median_values = df[numeric_cols].median()
df[numeric_cols] = df[numeric_cols].fillna(median_values)

In [19]:
# 4.3 Impute categorical/text features with mode
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

mode_values = {}
for c in cat_cols:
    # If all missing, leave as is; else fill with mode
    if df[c].notna().any():
        mode_values[c] = df[c].mode(dropna=True).iloc[0]
        df[c] = df[c].fillna(mode_values[c])

## 3.5 Stage 5 - Sanity Checks & Data Quality Report

**Role:**  
Verify cleaning worked as intended.

**Key Actions:**
- Recount missing values after imputation.  
- Verify target distribution and key ranges.  
- Assert no impossible values remain.  

In [20]:
def check_ranges(_df):
    msgs = []

    if 'Age' in _df.columns:
        bad = _df[( _df['Age'] < 0 ) | (_df['Age'] > 120)]
        msgs.append(f'Age out-of-range rows: {len(bad)}')

    if 'BMI' in _df.columns:
        bad = _df[( _df['BMI'] < 10 ) | (_df['BMI'] > 90)]
        msgs.append(f'BMI out-of-range rows: {len(bad)}')

    for day_col in ['Mental (days)','Physical (days)']:
        if day_col in _df.columns:
            bad = _df[( _df[day_col] < 0 ) | (_df[day_col] > 30)]
            msgs.append(f'{day_col} out-of-range rows: {len(bad)}')

    return "\n".join(msgs)

print(check_ranges(df))

Age out-of-range rows: 0
BMI out-of-range rows: 0
Mental (days) out-of-range rows: 0
Physical (days) out-of-range rows: 0


In [21]:
# Target distribution overview
if 'Diabetes' in df.columns:
    print("Target class distribution (original):")
    print(df['Diabetes'].value_counts(dropna=False))
if 'Diabetes_label' in df.columns:
    print("\nTarget label distribution (encoded):")
    print(df['Diabetes_label'].value_counts(dropna=False))

Target class distribution (original):
Diabetes
No             205133
Diabetes        34026
Prediabetes      4459
Name: count, dtype: int64

Target label distribution (encoded):
Diabetes_label
0.0    205133
2.0     34026
1.0      4459
Name: count, dtype: int64


In [22]:
# Quick info table
final_info = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'n_missing': df.isna().sum(),
    'missing_%': (df.isna().sum() / len(df) * 100).round(2)
}).sort_values('missing_%', ascending=False)

final_info

,dtype,n_missing,missing_%
CholCheck,float64,0,0.0
BloodPressure,float64,0,0.0
Alcoholic,float64,0,0.0
Income_num,float64,0,0.0
GeneralHealth_ord,float64,0,0.0
Education_ord,float64,0,0.0
Sex_bin,float64,0,0.0
Diabetes_label,float64,0,0.0
CholCheck_missing,int64,0,0.0
Stroke_missing,int64,0,0.0


## 3.6 Stage 6 - Save Final Clean File

**Role:**  
Produce the artifact to feed into **EDA** and **Modeling**.  

**Outcome:**  
`diabetes_diagnosis_clean.csv`  

In [30]:
df.to_csv(CLEAN_PATH, index=False)
print(f"Saved the cleaned dataset to :{CLEAN_PATH}")

Saved the cleaned dataset to :diabetes_diagnosis_clean.csv
